<a href="https://colab.research.google.com/github/dimitarpg13/agentic_architectures_and_design_patterns/blob/main/notebooks/live_web_search/duckduckgo_langgraph_mcp_multiagent_ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Agent Web Search System with MCP Protocol and LangGraph

This notebook demonstrates an advanced multi-agent system that uses:
- **MCP (Model Context Protocol)**: Standardized communication between agents
- **LangGraph**: Agent orchestration and workflow management
- **DuckDuckGo**: Live web search capabilities
- **Multi-Agent Design**: Specialized agents working together

## Architecture Overview

```
User Query → Coordinator Agent → Research Agent (DuckDuckGo)
                              ↓
                         Analyzer Agent
                              ↓
                        Synthesizer Agent → Final Response
```

### Agent Roles:
1. **Coordinator**: Routes queries and manages workflow
2. **Researcher**: Performs web searches using DuckDuckGo
3. **Analyzer**: Evaluates and validates search results
4. **Synthesizer**: Combines findings into coherent response

## 1. Install Required Packages

In [ ]:
# Install required packages
!pip install -q langchain langchain-openai langgraph duckduckgo-search langchain-community pydantic asyncio

## 2. Import Dependencies

In [ ]:
import os
import json
from typing import TypedDict, Annotated, Literal, List, Dict, Any
from datetime import datetime
import operator

from langchain_community.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

print("✅ All imports successful!")

## 3. Set Up API Keys

In [ ]:
# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Or load from environment
# from dotenv import load_dotenv
# load_dotenv()

## 4. Define MCP Protocol Structures

MCP (Model Context Protocol) defines standardized message formats for agent communication.

In [ ]:
# MCP Message Types
class MCPMessage(BaseModel):
    """Base MCP message structure."""
    message_id: str
    timestamp: str
    sender: str
    receiver: str
    message_type: str
    content: Dict[str, Any]
    metadata: Dict[str, Any] = Field(default_factory=dict)

class MCPSearchRequest(BaseModel):
    """MCP request for web search."""
    query: str
    num_results: int = 5
    search_type: Literal["general", "news", "scholarly"] = "general"

class MCPSearchResult(BaseModel):
    """MCP formatted search result."""
    title: str
    snippet: str
    url: str
    relevance_score: float = 0.0

class MCPAnalysis(BaseModel):
    """MCP analysis result."""
    summary: str
    key_points: List[str]
    credibility_score: float
    sources_count: int

class MCPSynthesis(BaseModel):
    """MCP final synthesis."""
    answer: str
    confidence: float
    sources: List[str]
    reasoning: str

print("✅ MCP Protocol structures defined!")

## 5. Define Multi-Agent State

In [ ]:
class MultiAgentState(TypedDict):
    """State shared across all agents."""
    # Original query
    query: str
    
    # MCP messages between agents
    mcp_messages: Annotated[List[Dict], operator.add]
    
    # Agent outputs
    search_results: List[Dict]
    analysis: Dict[str, Any]
    synthesis: Dict[str, Any]
    
    # Workflow control
    current_agent: str
    next_agent: str
    iteration: int
    
    # Final output
    final_response: str

print("✅ Multi-agent state defined!")

## 6. Create MCP Message Helper Functions

In [ ]:
import uuid

def create_mcp_message(
    sender: str,
    receiver: str,
    message_type: str,
    content: Dict[str, Any],
    metadata: Dict[str, Any] = None
) -> Dict:
    """Create a standardized MCP message."""
    message = MCPMessage(
        message_id=str(uuid.uuid4()),
        timestamp=datetime.now().isoformat(),
        sender=sender,
        receiver=receiver,
        message_type=message_type,
        content=content,
        metadata=metadata or {}
    )
    return message.model_dump()

def log_mcp_message(message: Dict):
    """Log MCP message for debugging."""
    print(f"\n📨 MCP Message:")
    print(f"  From: {message['sender']} → To: {message['receiver']}")
    print(f"  Type: {message['message_type']}")
    print(f"  Time: {message['timestamp']}")
    print(f"  Content: {json.dumps(message['content'], indent=2)[:200]}...")

print("✅ MCP helper functions ready!")

## 7. Agent 1: Coordinator Agent

In [ ]:
class CoordinatorAgent:
    """Coordinates the multi-agent workflow."""
    
    def __init__(self, llm):
        self.llm = llm
        self.name = "coordinator"
        
    def process(self, state: MultiAgentState) -> MultiAgentState:
        """Analyze query and route to appropriate agents."""
        print("\n" + "="*60)
        print("🎯 COORDINATOR AGENT")
        print("="*60)
        
        query = state["query"]
        
        # Analyze the query
        prompt = f"""Analyze this query and determine the search strategy:
        
Query: {query}

Provide:
1. Search keywords (comma-separated)
2. Number of results needed (1-10)
3. Search type (general/news/scholarly)
4. Complexity (simple/moderate/complex)

Return as JSON with keys: keywords, num_results, search_type, complexity"""
        
        response = self.llm.invoke([HumanMessage(content=prompt)])
        
        # Parse response (simplified)
        try:
            analysis = json.loads(response.content)
        except:
            analysis = {
                "keywords": query,
                "num_results": 5,
                "search_type": "general",
                "complexity": "moderate"
            }
        
        print(f"📋 Query Analysis: {analysis}")
        
        # Create MCP message for researcher
        mcp_msg = create_mcp_message(
            sender="coordinator",
            receiver="researcher",
            message_type="search_request",
            content={
                "query": analysis["keywords"],
                "num_results": analysis["num_results"],
                "search_type": analysis["search_type"]
            },
            metadata={"complexity": analysis["complexity"]}
        )
        
        log_mcp_message(mcp_msg)
        
        return {
            "mcp_messages": [mcp_msg],
            "current_agent": "coordinator",
            "next_agent": "researcher",
            "iteration": state.get("iteration", 0) + 1
        }

print("✅ Coordinator Agent created!")

## 8. Agent 2: Researcher Agent (with DuckDuckGo)

In [ ]:
class ResearcherAgent:
    """Performs web searches using DuckDuckGo."""
    
    def __init__(self):
        self.name = "researcher"
        self.search_tool = DuckDuckGoSearchResults(num_results=5)
        
    def process(self, state: MultiAgentState) -> MultiAgentState:
        """Execute web search based on MCP request."""
        print("\n" + "="*60)
        print("🔍 RESEARCHER AGENT")
        print("="*60)
        
        # Get the latest MCP message
        mcp_messages = state.get("mcp_messages", [])
        if not mcp_messages:
            return state
            
        last_message = mcp_messages[-1]
        search_request = last_message["content"]
        
        print(f"🔎 Searching for: {search_request['query']}")
        
        # Perform search
        try:
            raw_results = self.search_tool.run(search_request["query"])
            
            # Parse results
            if isinstance(raw_results, str):
                # Parse string results
                import re
                results = []
                snippets = raw_results.split('snippet: ')
                
                for i, snippet in enumerate(snippets[1:], 1):  # Skip first empty split
                    parts = snippet.split('title: ')
                    if len(parts) > 1:
                        title = parts[1].split('link: ')[0].strip()
                        link = parts[1].split('link: ')[1].strip() if 'link: ' in parts[1] else ""
                        results.append({
                            "title": title[:100],
                            "snippet": parts[0].strip()[:200],
                            "url": link,
                            "relevance_score": 1.0 - (i * 0.1)
                        })
            else:
                results = raw_results[:search_request["num_results"]]
                
            print(f"✅ Found {len(results)} results")
            
            # Create MCP message with results
            mcp_msg = create_mcp_message(
                sender="researcher",
                receiver="analyzer",
                message_type="search_results",
                content={
                    "results": results,
                    "query": search_request["query"],
                    "results_count": len(results)
                }
            )
            
            log_mcp_message(mcp_msg)
            
            return {
                "search_results": results,
                "mcp_messages": [mcp_msg],
                "current_agent": "researcher",
                "next_agent": "analyzer"
            }
            
        except Exception as e:
            print(f"❌ Search error: {e}")
            return {
                "search_results": [],
                "current_agent": "researcher",
                "next_agent": "analyzer"
            }

print("✅ Researcher Agent created!")

## 9. Agent 3: Analyzer Agent

In [ ]:
class AnalyzerAgent:
    """Analyzes and validates search results."""
    
    def __init__(self, llm):
        self.llm = llm
        self.name = "analyzer"
        
    def process(self, state: MultiAgentState) -> MultiAgentState:
        """Analyze search results quality and relevance."""
        print("\n" + "="*60)
        print("📊 ANALYZER AGENT")
        print("="*60)
        
        search_results = state.get("search_results", [])
        query = state["query"]
        
        if not search_results:
            print("⚠️  No search results to analyze")
            return state
        
        # Prepare results summary for analysis
        results_summary = "\n".join([
            f"{i+1}. {r.get('title', 'N/A')}: {r.get('snippet', 'N/A')[:100]}"
            for i, r in enumerate(search_results[:5])
        ])
        
        prompt = f"""Analyze these search results for the query: "{query}"

Results:
{results_summary}

Provide:
1. Summary of findings (2-3 sentences)
2. Key points (3-5 bullet points)
3. Credibility score (0-1)
4. Are results sufficient? (yes/no)

Return as JSON with keys: summary, key_points (array), credibility_score, sufficient"""
        
        response = self.llm.invoke([HumanMessage(content=prompt)])
        
        # Parse analysis
        try:
            analysis = json.loads(response.content)
        except:
            analysis = {
                "summary": "Analysis of search results completed.",
                "key_points": ["Multiple sources found", "Relevant information available"],
                "credibility_score": 0.7,
                "sufficient": True
            }
        
        print(f"📈 Credibility Score: {analysis.get('credibility_score', 0)}")
        print(f"✓ Sufficient: {analysis.get('sufficient', False)}")
        
        # Create MCP message
        mcp_msg = create_mcp_message(
            sender="analyzer",
            receiver="synthesizer",
            message_type="analysis_results",
            content=analysis
        )
        
        log_mcp_message(mcp_msg)
        
        return {
            "analysis": analysis,
            "mcp_messages": [mcp_msg],
            "current_agent": "analyzer",
            "next_agent": "synthesizer"
        }

print("✅ Analyzer Agent created!")

## 10. Agent 4: Synthesizer Agent

In [ ]:
class SynthesizerAgent:
    """Synthesizes final response from all agent outputs."""
    
    def __init__(self, llm):
        self.llm = llm
        self.name = "synthesizer"
        
    def process(self, state: MultiAgentState) -> MultiAgentState:
        """Create final comprehensive response."""
        print("\n" + "="*60)
        print("🎨 SYNTHESIZER AGENT")
        print("="*60)
        
        query = state["query"]
        search_results = state.get("search_results", [])
        analysis = state.get("analysis", {})
        
        # Prepare context
        context = f"""Original Query: {query}

Search Results Summary:
{analysis.get('summary', 'No analysis available')}

Key Points:
{chr(10).join('- ' + point for point in analysis.get('key_points', []))}

Top Sources:
{chr(10).join(f"{i+1}. {r.get('title', 'N/A')} - {r.get('url', 'N/A')}" for i, r in enumerate(search_results[:3]))}
"""
        
        prompt = f"""Based on the following information, create a comprehensive answer:

{context}

Provide:
1. A clear, well-structured answer to the query
2. Confidence level (0-1)
3. Reasoning for your answer

Return as JSON with keys: answer, confidence, reasoning"""
        
        response = self.llm.invoke([HumanMessage(content=prompt)])
        
        # Parse synthesis
        try:
            synthesis = json.loads(response.content)
        except:
            synthesis = {
                "answer": response.content,
                "confidence": 0.7,
                "reasoning": "Based on web search results"
            }
        
        print(f"✨ Confidence: {synthesis.get('confidence', 0)}")
        
        # Create final MCP message
        mcp_msg = create_mcp_message(
            sender="synthesizer",
            receiver="coordinator",
            message_type="final_response",
            content={
                "answer": synthesis["answer"],
                "confidence": synthesis["confidence"],
                "sources": [r.get("url", "") for r in search_results[:3]]
            }
        )
        
        log_mcp_message(mcp_msg)
        
        # Format final response
        final_response = f"""**Answer:**
{synthesis['answer']}

**Confidence:** {synthesis['confidence']:.0%}

**Sources:**
{chr(10).join(f"- {url}" for url in [r.get('url', '') for r in search_results[:3]] if url)}

**Reasoning:** {synthesis.get('reasoning', 'N/A')}
"""
        
        return {
            "synthesis": synthesis,
            "mcp_messages": [mcp_msg],
            "final_response": final_response,
            "current_agent": "synthesizer",
            "next_agent": "end"
        }

print("✅ Synthesizer Agent created!")

## 11. Build the Multi-Agent LangGraph Workflow

In [ ]:
# Initialize LLM
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Initialize agents
coordinator = CoordinatorAgent(llm)
researcher = ResearcherAgent()
analyzer = AnalyzerAgent(llm)
synthesizer = SynthesizerAgent(llm)

# Create workflow nodes
def coordinator_node(state: MultiAgentState) -> MultiAgentState:
    return coordinator.process(state)

def researcher_node(state: MultiAgentState) -> MultiAgentState:
    return researcher.process(state)

def analyzer_node(state: MultiAgentState) -> MultiAgentState:
    return analyzer.process(state)

def synthesizer_node(state: MultiAgentState) -> MultiAgentState:
    return synthesizer.process(state)

# Build the graph
workflow = StateGraph(MultiAgentState)

# Add nodes
workflow.add_node("coordinator", coordinator_node)
workflow.add_node("researcher", researcher_node)
workflow.add_node("analyzer", analyzer_node)
workflow.add_node("synthesizer", synthesizer_node)

# Define edges (workflow sequence)
workflow.set_entry_point("coordinator")
workflow.add_edge("coordinator", "researcher")
workflow.add_edge("researcher", "analyzer")
workflow.add_edge("analyzer", "synthesizer")
workflow.add_edge("synthesizer", END)

# Compile the graph
app = workflow.compile()

print("\n✅ Multi-Agent System Ready!")
print("\n🤖 Agent Pipeline: Coordinator → Researcher → Analyzer → Synthesizer")

## 12. Visualize the Multi-Agent Workflow

In [ ]:
# Visualize the workflow
try:
    from IPython.display import Image, display
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not visualize graph: {e}")
    print("\nWorkflow Structure:")
    print("Entry → Coordinator → Researcher → Analyzer → Synthesizer → End")

## 13. Helper Function to Run Multi-Agent System

In [ ]:
def run_multi_agent_search(query: str) -> str:
    """
    Run the multi-agent search system.
    
    Args:
        query: User's search query
        
    Returns:
        Final synthesized response
    """
    print("\n" + "="*80)
    print("🚀 MULTI-AGENT WEB SEARCH SYSTEM")
    print("="*80)
    print(f"\n❓ Query: {query}\n")
    
    # Initialize state
    initial_state = {
        "query": query,
        "mcp_messages": [],
        "search_results": [],
        "analysis": {},
        "synthesis": {},
        "current_agent": "start",
        "next_agent": "coordinator",
        "iteration": 0,
        "final_response": ""
    }
    
    # Run the workflow
    try:
        final_state = app.invoke(initial_state)
        
        print("\n" + "="*80)
        print("✅ FINAL RESPONSE")
        print("="*80)
        print(final_state["final_response"])
        
        # Display MCP message summary
        print("\n" + "="*80)
        print("📨 MCP MESSAGE SUMMARY")
        print("="*80)
        for i, msg in enumerate(final_state.get("mcp_messages", []), 1):
            print(f"{i}. {msg['sender']} → {msg['receiver']}: {msg['message_type']}")
        
        return final_state["final_response"]
        
    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return f"Error processing query: {e}"

print("✅ Helper function ready!")

## 14. Example 1: Current Events Query

In [ ]:
# Example 1: Current events
response1 = run_multi_agent_search(
    "What are the latest breakthroughs in quantum computing?"
)

## 15. Example 2: Technical Comparison

In [ ]:
# Example 2: Technical comparison
response2 = run_multi_agent_search(
    "Compare LangChain vs LlamaIndex for building LLM applications"
)

## 16. Example 3: Complex Research Question

In [ ]:
# Example 3: Complex research
response3 = run_multi_agent_search(
    "What is the environmental impact of AI model training and what solutions are being proposed?"
)

## 17. Advanced: Adding Feedback Loops

In [ ]:
# Advanced workflow with feedback loops
class AdaptiveMultiAgentState(MultiAgentState):
    """Extended state with feedback loop support."""
    needs_more_research: bool
    research_iterations: int
    max_iterations: int

def should_continue_research(state: AdaptiveMultiAgentState) -> str:
    """Determine if more research is needed."""
    analysis = state.get("analysis", {})
    iterations = state.get("research_iterations", 0)
    max_iter = state.get("max_iterations", 2)
    
    # Check if results are sufficient
    sufficient = analysis.get("sufficient", True)
    credibility = analysis.get("credibility_score", 1.0)
    
    if not sufficient and credibility < 0.7 and iterations < max_iter:
        print("🔄 Feedback Loop: More research needed")
        return "researcher"  # Go back to researcher
    else:
        return "synthesizer"  # Proceed to synthesis

# Build adaptive workflow (example structure)
adaptive_workflow = StateGraph(AdaptiveMultiAgentState)
adaptive_workflow.add_node("coordinator", coordinator_node)
adaptive_workflow.add_node("researcher", researcher_node)
adaptive_workflow.add_node("analyzer", analyzer_node)
adaptive_workflow.add_node("synthesizer", synthesizer_node)

adaptive_workflow.set_entry_point("coordinator")
adaptive_workflow.add_edge("coordinator", "researcher")
adaptive_workflow.add_edge("researcher", "analyzer")

# Conditional edge with feedback loop
adaptive_workflow.add_conditional_edges(
    "analyzer",
    should_continue_research,
    {
        "researcher": "researcher",  # Loop back
        "synthesizer": "synthesizer"  # Continue forward
    }
)

adaptive_workflow.add_edge("synthesizer", END)

print("✅ Adaptive workflow with feedback loops created!")

## 18. MCP Protocol Monitoring Dashboard

In [ ]:
def display_mcp_analytics(state: MultiAgentState):
    """Display analytics on MCP message flow."""
    messages = state.get("mcp_messages", [])
    
    print("\n" + "="*80)
    print("📊 MCP PROTOCOL ANALYTICS")
    print("="*80)
    
    # Message count by type
    message_types = {}
    for msg in messages:
        msg_type = msg["message_type"]
        message_types[msg_type] = message_types.get(msg_type, 0) + 1
    
    print("\n📨 Message Types:")
    for msg_type, count in message_types.items():
        print(f"  - {msg_type}: {count}")
    
    # Communication flow
    print("\n🔄 Communication Flow:")
    for i, msg in enumerate(messages, 1):
        print(f"  {i}. {msg['sender']} → {msg['receiver']} ({msg['message_type']})")
    
    # Timing analysis
    print("\n⏱️  Timing:")
    if messages:
        first_time = datetime.fromisoformat(messages[0]["timestamp"])
        last_time = datetime.fromisoformat(messages[-1]["timestamp"])
        duration = (last_time - first_time).total_seconds()
        print(f"  Total Duration: {duration:.2f} seconds")
        print(f"  Avg Time/Message: {duration/len(messages):.2f} seconds")

print("✅ MCP analytics function ready!")

## 19. Run Complete System with Analytics

In [ ]:
# Run with full analytics
query = "What are the ethical considerations of AI in healthcare?"

initial_state = {
    "query": query,
    "mcp_messages": [],
    "search_results": [],
    "analysis": {},
    "synthesis": {},
    "current_agent": "start",
    "next_agent": "coordinator",
    "iteration": 0,
    "final_response": ""
}

final_state = app.invoke(initial_state)

# Display analytics
display_mcp_analytics(final_state)

## 20. Interactive Multi-Agent Chat

In [ ]:
# Interactive mode
def interactive_multi_agent_search():
    """Interactive mode for multi-agent search."""
    print("\n" + "="*80)
    print("🤖 MULTI-AGENT SEARCH SYSTEM - Interactive Mode")
    print("="*80)
    print("Type 'quit' to exit, 'analytics' to see MCP stats\n")
    
    conversation_history = []
    
    while True:
        query = input("\n❓ Your question: ")
        
        if query.lower() in ['quit', 'exit', 'q']:
            print("\n👋 Goodbye!")
            break
        
        if query.lower() == 'analytics' and conversation_history:
            display_mcp_analytics(conversation_history[-1])
            continue
        
        if not query.strip():
            continue
        
        # Run search
        initial_state = {
            "query": query,
            "mcp_messages": [],
            "search_results": [],
            "analysis": {},
            "synthesis": {},
            "current_agent": "start",
            "next_agent": "coordinator",
            "iteration": 0,
            "final_response": ""
        }
        
        try:
            final_state = app.invoke(initial_state)
            conversation_history.append(final_state)
            print("\n" + final_state["final_response"])
        except Exception as e:
            print(f"\n❌ Error: {e}")

# Uncomment to run interactive mode
# interactive_multi_agent_search()

## 21. Best Practices and Optimization Tips

### MCP Protocol Benefits:
1. **Standardization**: Consistent message format across agents
2. **Traceability**: Full audit trail of agent communications
3. **Debugging**: Easy to identify where issues occur
4. **Extensibility**: Simple to add new agents or message types

### Multi-Agent Design Principles:
1. **Separation of Concerns**: Each agent has a specific role
2. **Loose Coupling**: Agents communicate via standardized messages
3. **Scalability**: Easy to parallelize or distribute agents
4. **Fault Tolerance**: One agent failure doesn't crash the system

### Performance Optimization:
1. **Caching**: Cache search results to avoid redundant queries
2. **Parallel Processing**: Run independent agents in parallel
3. **Timeout Handling**: Set timeouts for each agent
4. **Result Streaming**: Stream results as they become available

### Production Considerations:
1. **Error Handling**: Implement comprehensive error handling
2. **Monitoring**: Track agent performance and MCP message flow
3. **Rate Limiting**: Respect API rate limits (especially for search)
4. **Testing**: Unit test each agent independently
5. **Logging**: Detailed logging for debugging and analytics

## 22. Extending the System

### Additional Agents to Consider:
1. **Fact Checker Agent**: Verifies claims against multiple sources
2. **Citation Agent**: Properly formats and tracks sources
3. **Translation Agent**: Handles multilingual queries
4. **Summarization Agent**: Creates different summary lengths
5. **Bias Detection Agent**: Identifies potential bias in sources

### Alternative Search Providers:
```python
# Brave Search (requires API key)
from langchain_community.tools import BraveSearch

# Tavily (optimized for LLMs)
from langchain_community.tools.tavily_search import TavilySearchResults

# SerpAPI (Google search)
from langchain_community.utilities import SerpAPIWrapper
```

### Enhanced MCP Features:
1. **Priority Queue**: Prioritize urgent messages
2. **Message Retry**: Automatic retry on failure
3. **Message Validation**: Schema validation for all messages
4. **Message Routing**: Dynamic routing based on content

## 23. Resources and Further Learning

### Documentation:
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [LangChain Multi-Agent Systems](https://python.langchain.com/docs/use_cases/more/agents/multi_agent/)
- [Model Context Protocol Spec](https://modelcontextprotocol.io/)

### Related Topics:
- Agent architectures (ReAct, Plan-and-Execute)
- Tool calling and function schemas
- Prompt engineering for agents
- Distributed systems patterns

### Next Steps:
1. Add memory/persistence to agents
2. Implement parallel agent execution
3. Create a web UI with Streamlit
4. Deploy with FastAPI backend
5. Add authentication and rate limiting